In [20]:
##Let's first calculate the vector of each movie from 1999
import pandas as pd
import numpy as np
omdf = pd.read_csv("./1999_movies_genre_profit.tsv", sep='\t')

genre_to_id = {
    'Drama': 0, 'Horror': 1, 'Action': 2, 'Crime': 3, 'Thriller': 4, 
    'Family': 5, 'Sci-Fi': 6, 'Comedy': 7, 'Romance': 8, 'War': 9, 
    'Adventure': 10, 'Fantasy': 11, 'Animation': 12, 'Mystery': 13, 
    'History': 14, 'Western': 15, 'Music': 16, 'Biography': 17, 
    'Sport': 18, 'Musical': 19, 'Documentary': 20
}
omdf['genres_vector'] = [np.zeros(21) for _ in range(len(omdf))]

for index, row in omdf.iterrows():
    genres = row['genres'].split(',')
    genres_vector = np.zeros(21)
    for genre in genres:
        genres_vector[genre_to_id[genre]] = 1
    omdf.at[index, 'genres_vector'] = genres_vector


In [21]:
## Let's now calculate the vector of each movie in the final dataset
from tqdm import tqdm
mdf = pd.read_csv("../Final/rly_final_movies.tsv", sep='\t')
mdf['genres_vector'] = [np.zeros(21) for _ in range(len(mdf))]

for index, row in tqdm(mdf.iterrows()):
    genres = row['genres'].split(',')
    genres_vector = np.zeros(21)
    for genre in genres:
        genres_vector[genre_to_id[genre]] = 1
    mdf.at[index, 'genres_vector'] = genres_vector

5477it [00:00, 18817.35it/s]


In [27]:
## Let's calculate our new feature
mdf["profit"] = mdf["revenue"] - mdf["budget"]
mdf['weighted_genre_profitability'] = 0

In [29]:
##Starting from movies 2000
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

for index, row in tqdm(mdf.iterrows()):
    if row["startYear"] > 2000:
        continue
    result = 0
    for i,r in omdf.iterrows():
        result += (cosine_similarity(row["genres_vector"], r["genres_vector"]) * r["profit"])
    mdf.at[index, "weighted_genre_profitability"] = result

0it [00:00, ?it/s]C:\Users\moham\AppData\Local\Temp\ipykernel_16860\3622195200.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1919887183.2597163' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  mdf.at[index, "weighted_genre_profitability"] = result
5477it [00:02, 2300.47it/s]


In [31]:
## Other movies
for index, row in tqdm(mdf.iterrows()):
    if row["startYear"] <= 2000:
        continue
    last_year = row["startYear"] - 1 
    result = 0
    for i,r in mdf[mdf["startYear"] == last_year].iterrows():
        result += (cosine_similarity(row["genres_vector"], r["genres_vector"]) * r["profit"])
    mdf.at[index, "weighted_genre_profitability"] = result

5477it [01:06, 82.45it/s] 


In [33]:
mdf.to_csv("../Final/rly_final_movies.tsv", sep='\t', index=False)